In [1]:
print("hello")

hello


In [ ]:
%pwd

'c:\\Users\\nakul\\medico\\research'

In [2]:
import os
os.chdir("../")

In [3]:
from pypdf import PdfReader

In [6]:
reader = PdfReader('data/Medical_book.pdf')

In [7]:
print(len(reader.pages))

637


In [8]:
page = reader.pages[0]
print(page.extract_text())

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

c:\Users\nakul\anaconda3\envs\medico\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
docs = load_pdf('data')

In [7]:
def text_splitter(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, 
        chunk_overlap=20)
    text_chunks = text_splitter.split_documents(docs)
    return text_chunks

In [8]:
text_chunk = text_splitter(docs)

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [20]:
def download_embedding():
      model_name = "sentence-transformers/all-MiniLM-l6-v2"
      embeddings = HuggingFaceEmbeddings(
            model_name= model_name)
      return embeddings

embeddings = download_embedding()

C:\Users\nakul\AppData\Local\Temp\ipykernel_24540\2703632793.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [11]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [12]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [13]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key = pinecone_api_key)

In [14]:
pc

In [18]:
from pinecone import ServerlessSpec

index_name = 'medico'

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension = 384,
        metric= 'cosine',
        spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1')
    )
index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunk,
    embedding = embeddings,
    index_name = index_name
)

In [21]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [22]:
retriver = docsearch.as_retriever(search_type= "similarity", search_kwargs = {"k" : 3})

In [25]:
retrived = retriver.invoke("what is acne?")
retrived

[Document(id='e44caf4b-9bfc-4426-9329-cb323324fd3c', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='ca715c4e-20d0-4b5b-8c9d-9e2a823f8b3c', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM 

In [27]:
from langchain_openai import ChatOpenAI

In [28]:
chatModel = ChatOpenAI(
    model = "gpt-4o"
)

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [38]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder characterized by abnormal release of a growth hormone from the pituitary gland, leading to increased growth in bone and soft tissue and various body disturbances. It occurs when the abnormality arises after bone growth has stopped. Gigantism, in contrast, happens when this abnormal growth occurs before bone growth stops, leading to unusual height.
